In [1]:
import numpy as np
import pandas as pd
import math
# to get logistic regression, grab LogisticRegression from linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

#### Logistic Regression
- Slide Deck: 
https://www.canva.com/design/DAEfWtGKNgc/VqwX9iLNaf4TV7YJg6XflQ/view?utm_content=DAEfWtGKNgc&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton

- logistic regression in sklearn

In [2]:
from pydataset import data

df = data('iris')
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa


In [3]:
# columns name change
df.columns = [col.lower().replace('.', '_') for col in df]
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [4]:
# Binary classification - predict if species is non-virginica or virginica
# change setosa and versicolor to '0' and virginica to 1
df['species'] = np.where(df.species == 'virginica', 1, 0)

In [5]:
df.species.value_counts()

0    100
1     50
Name: species, dtype: int64

In [6]:
df.head(2)

,sepal_length,sepal_width,petal_length,petal_width,species
1,5.1,3.5,1.4,0.2,0
2,4.9,3.0,1.4,0.2,0


## Predict if species is virginica or not

In [7]:
def train_validate_test_split(df, target, seed=1349):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=df[target],)
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test

In [8]:
train, validate, test = train_validate_test_split(df,
                                                  target='species',
                                                  seed=1349)

In [9]:
train.shape, validate.shape, test.shape

((84, 5), (36, 5), (30, 5))

In [10]:
# Make new dataframes
# Separate my X and my y! 

X_train = train.drop(columns='species')
y_train = train.species
X_validate = validate.drop(columns='species')
y_validate = validate.species
X_test = test.drop(columns='species')
y_test = test.species

In [11]:
X_train

,sepal_length,sepal_width,petal_length,petal_width
124,6.3,2.7,4.9,1.8
96,5.7,3.0,4.2,1.2
143,5.8,2.7,5.1,1.9
92,6.1,3.0,4.6,1.4
122,5.6,2.8,4.9,2.0
21,5.4,3.4,1.7,0.2
79,6.0,2.9,4.5,1.5
120,6.0,2.2,5.0,1.5
38,4.9,3.6,1.4,0.1
44,5.0,3.5,1.6,0.6


# Model 1

In [12]:
# make, fit, use!

In [13]:
# Define the logistic regression model
logit = LogisticRegression()

In [14]:
#  fit the model on train data
logit.fit(X_train, y_train)

LogisticRegression()

In [15]:
# now use the model to make predictions
y_pred = logit.predict(X_train)

In [16]:
#take a look at predictions
y_pred[:5]

array([1, 0, 1, 0, 1])

In [29]:
# look at predicted probabilites for first 10 observations
# we can tune the probability threshold very explicitly here if we so choose to,
# by narrowing down to the positive class and comaparing values
# to desired probability thresholds
# default for predict inside of sklearn objects is generally 0.5
logit.predict_proba(X_train)[:10][:,1] > 0.5

array([ True, False,  True, False,  True, False, False,  True, False,
       False])

In [28]:
logit.classes_

array([0, 1])

In [30]:
# View raw probabilities (output from the model)
y_pred_proba = logit.predict_proba(X_train)
y_pred_proba = pd.DataFrame(y_pred_proba,
columns = ['non-virginica', 'virginica'])



In [32]:
y_pred_proba.head(5)

,non-virginica,virginica
0,0.377272,0.622728
1,0.923081,0.076919
2,0.221020,0.778980
3,0.757160,0.242840
4,0.273458,0.726542


In [33]:
# classification report
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00        28

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84



In [34]:
logit.coef_

array([[-0.12099641, -0.29704264,  2.43792504,  2.10506578]])

In [ ]:
# 1/e^-(B0x + B1(x1).... etc)

## Model 2

In [35]:
# Change hyperparameter C = 0.01
logit2 = LogisticRegression(C=.01, random_state=1349)

In [36]:
# fit the model
logit2.fit(X_train, y_train)

LogisticRegression(C=0.01, random_state=1349)

In [37]:
# make prediction
y_pred2 = logit2.predict(X_train)

In [38]:
#classification report
print(classification_report(y_train, y_pred2))

              precision    recall  f1-score   support

           0       0.73      1.00      0.84        56
           1       1.00      0.25      0.40        28

    accuracy                           0.75        84
   macro avg       0.86      0.62      0.62        84
weighted avg       0.82      0.75      0.69        84



## Evaluate Model 1 and 2 performance on 'Validate'

In [41]:
# Make prediction for validate dataset

In [40]:
print(classification_report(y_validate, logit.predict(X_validate)))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98        24
           1       0.92      1.00      0.96        12

    accuracy                           0.97        36
   macro avg       0.96      0.98      0.97        36
weighted avg       0.97      0.97      0.97        36



In [42]:
print(classification_report(y_validate, logit2.predict(X_validate)))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91        24
           1       1.00      0.58      0.74        12

    accuracy                           0.86        36
   macro avg       0.91      0.79      0.82        36
weighted avg       0.89      0.86      0.85        36



### Hyperparameters
#### Regularization:
- Keep model simple
- Constraints the coefficients
- Discourages learning more complex model
- Minimizes overfitting
- L1 - Lasso
- L2 - Ridge

#### C = Inverse of regularization strength:

- Lower C - higher regularization
- Lower C discourages learning more complex model
- minimizes overfitting

## Bonus: Interpreting model coefficients

In [47]:
# look at model 1 coefficents
logit.coef_[0]


array([-0.12099641, -0.29704264,  2.43792504,  2.10506578])

#### Logistic Regression basics:

log(odds) = log(p/(1-p)) = $intercept$ + ($\beta_1$ * variable1) + ($\beta_2$ * variable2) + ($\beta_3$ * variable3)

**The coefficients above represents 'log odds'**

In [48]:
# Make a dataframe of coefficients and feature names

log_coeffs = pd.DataFrame(logit.coef_[0], index=X_train.columns,
                         columns=['coeff'])
log_coeffs

,coeff
sepal_length,-0.120996
sepal_width,-0.297043
petal_length,2.437925
petal_width,2.105066


**It would be helpful to convert 'log odds' to 'odds'**

In [49]:
# convert from log odds to odds (exponentiate)
odds = np.exp(log_coeffs)

In [50]:
odds

,coeff
sepal_length,0.886037
sepal_width,0.743012
petal_length,11.449259
petal_width,8.207643


What is odds?

odds = P(occurring) / P(not occurring)  = p / (1-p)

Toss a fair coin
odds = 0.5 / (1-0.5) = 1   i.e. Odd of landing tails vs heads is 1:1 for fair coin

Rolling 2 or higher on a dice roll  
odd = (5/6) /  (1/6) = 5 i.e. Odd of rolling a 2 or higher on a dice is 5:1 for a fair die

#### Coefficient Interpretation (odds):


- **Example: petal_length: For every one unit increase in petal_length, we expect 10 times increase in odds of being a 'virginica' vs a 'non-virginica'.**


- **If the coefficient (odds) is 1 or close to 1, this means odds of being in class '1' (positive class) is same or close to being in class '0' (negative class). This means the feature with this coefficient is not a big driver for the target variable in this particular model**

- **If the coefficient value is < 1 , that implies that increase in value of that feature will decrease the odds that target variable is in positive class**